In [1]:
from tkinter import *
from PIL import Image, ImageTk
import random

import pickle
import random

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras

from keras import backend as K
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import *
from tensorflow.python.keras.optimizers import *
from tensorflow.python.keras.callbacks import ModelCheckpoint

print(tf.__version__)
print(keras.__version__)
print(TkVersion)

model = load_model('night_model.hdf5')
LETTERS = list('abcdefghjklmnop')
NUMBERS = [i for i in range(1, 16)]
starsSet = set([(i, j) for i in [3, 7, 11] for j in [3, 7, 11]])

pictureEmptyDir = 'images/empty.png'
pictureEmptyUpDir = 'images/up.png'
pictureEmptyDownDir = 'images/down.png'
pictureEmptyRightDir = 'images/right.png'
pictureEmptyLeftDir = 'images/left.png'
pictureEmptySpecialDir = 'images/empty_special2.png'
pictureEmptyNWDir = 'images/nw.png'
pictureEmptySWDir = 'images/sw.png'
pictureEmptySEDir = 'images/se.png'
pictureEmptyNEDir = 'images/ne.png'

pictureBlackDir = 'images/black.png'
pictureBlackUpDir = 'images/black_up.png'
pictureBlackDownDir = 'images/black_down.png'
pictureBlackRightDir = 'images/black_right.png'
pictureBlackLeftDir = 'images/black_left.png'
pictureBlackNWDir = 'images/blacknw.png'
pictureBlackSWDir = 'images/blacksw.png'
pictureBlackSEDir = 'images/blackse.png'
pictureBlackNEDir = 'images/blackne.png'

pictureWhiteDir = 'images/white.png'
pictureWhiteUpDir = 'images/white_up.png'
pictureWhiteDownDir = 'images/white_down.png'
pictureWhiteRightDir = 'images/white_right.png'
pictureWhiteLeftDir = 'images/white_left.png'
pictureWhiteNWDir = 'images/whitenw.png'
pictureWhiteSWDir = 'images/whitesw.png'
pictureWhiteSEDir = 'images/whitese.png'
pictureWhiteNEDir = 'images/whitene.png'

TABLE_WIDTH = 15
TABLE_HEIGHT = 15
CELL_WIDTH = 40
CELL_HEIGHT = 40
BUTTONS_NUM = 4

cells = [[0 for j in range(TABLE_WIDTH)] for i in range(TABLE_HEIGHT)]
buttons = [0 for i in range(BUTTONS_NUM)]
turn = 0

/Users/sergevkim/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)
Using TensorFlow backend.


1.12.0
2.2.4
8.6


In [2]:
def give_pic(string, column, color=0):
    if color == 0:
        kind = 'empty'
    elif color == 1:
        kind = 'black'
    else:
        kind = 'white'
        
    if string == 0:
        if column == 0:
            direct = 'NW'
        elif column == 14:
            direct = 'NE'
        else:
            direct = 'N'
    elif string == 14:
        if column == 0:
            direct = 'SW'
        elif column == 14:
            direct = 'SE'
        else:
            direct = 'S'
    elif column == 0:
        direct = 'W'
    elif column == 14:
        direct = 'E'
    elif (string, column) in starsSet:
        direct = 'Spec'
    else:
        direct = 'M'    # M is mid
        
    return kind + direct

def win_condition(color):
    for i in range(15):
        for j in range(11):
            counter = 0
            for k in range(5):
                if cells[i][j + k].color == color:
                    counter += 1
                elif cells[i][j + k].color == 'empty':
                    empty_cell_id = (i, j + k)
                else:
                    counter = 0
                    break
            if counter == 4:
                return empty_cell_id
    
    for j in range(15):
        for i in range(11):
            counter = 0
            for k in range(5):
                if cells[i + k][j].color == color:
                    counter += 1
                elif cells[i + k][j].color == 'empty':
                    empty_cell_id = (i + k, j)
                else:
                    counter = 0
                    break
            if counter == 4:
                return empty_cell_id
    
    for i in range(11):
        for j in range(11):
            counter = 0
            for k in range(5):
                if cells[i + k][j + k].color == color:
                    counter += 1
                elif cells[i + k][j + k].color == 'empty':
                    empty_cell_id = (i + k, j + k)
                else:
                    counter = 0
                    break
            if counter == 4:
                for k in range(5):
                    return empty_cell_id
                
    for i in range(11):
        for j in range(4, 15):
            counter = 0
            for k in range(5):
                if cells[i + k][j - k].color == color:
                    counter += 1
                elif cells[i + k][j - k].color == 'empty':
                    empty_cell_id = (i + k, j - k)
                else:
                    counter = 0
                    break
            if counter == 4:
                return empty_cell_id

    return (-1, -1)
                    

In [3]:
class Position:
    def __init__(self, turn, seq, label=None):
        self.fir = np.array([[0 for j in range(15)] for i in range(15)])
        self.sec = np.array([[0 for j in range(15)] for i in range(15)])
        self.thi = np.array([[turn for j in range(15)] for i in range(15)])
        self.label = label
        for i in range(len(seq)):
            if i % 2 == 0:
                self.fir[seq[i][0]][seq[i][1]] = 1
            else:
                self.sec[seq[i][0]][seq[i][1]] = -1
        self.image = np.array([self.fir, self.sec, self.thi])

class Texter:
    def __init__(self, button, text):
        self.button = button
        self.text = text

class BattleLog:
    def __init__(self, button, log, story=[]):
        self.button = button
        self.log = log
        self.story = story
        
class Cell:
    def __init__(self, x, y, button, color, picture=None):
        self.x = x
        self.y = y
        self.button = button
        self.color = color    # color in {'black', 'white', 'empty'}

In [4]:
# NO BUTTONS FOOS >>>>>>>

def translate(coords):    # type(coords) == str: k8, h1, etc. res is a tuple
    coordFir = LETTERS.index(coords[0])
    coordSec = int(coords[1:]) - 1
    return (coordSec, coordFir)

def translate2(coords):   #type(coords) == tuple: (7, 7), (9, 9), etc. res is number of class
    res = coords[0] * 15 + coords[1]
    return res

def to_seq(game):
    seq = []
    for i in range(len(game)):
        seq.append(translate(game[i]))
    return seq

# <<<<<<<<<<<<<<<<<




# BUTTONS FOOS >>>>>>>>>>>>>>>>

def predictShow():
    # >>> sometimes i need in this simple checking (one time to twenty)
    first_check = win_condition('black')
    second_check = win_condition('white')
    if first_check[0] != -1:
        answer_tuple = first_check
        answer_coords = LETTERS[answer_tuple[1]] + str(NUMBERS[answer_tuple[0]])
        buttons[3].text = answer_coords
        buttons[3].button.configure(text=answer_coords)
        return answer_coords
    if second_check[0] != -1:
        answer_tuple = second_check
        answer_coords = LETTERS[answer_tuple[1]] + str(NUMBERS[answer_tuple[0]])
        buttons[3].text = answer_coords
        buttons[3].button.configure(text=answer_coords)
        return answer_coords
    # <<< so why not
    
    forAnswer = buttons[1].story
    if turn % 2 == 0:
        curPlayer = 1
    else:
        curPlayer = -1
    pos = Position(turn=curPlayer, seq=to_seq(forAnswer))

    table_for_model = np.empty((15, 15, 3))
    for i in range(3):
        table_for_model[:,:,i] = pos.image[i,:,:]
        
    y = model.predict_proba(np.array([table_for_model]))
    answer_class = np.argmax(y)
    answer_tuple = (answer_class // 15, answer_class % 15)
    answer_coords = LETTERS[answer_tuple[1]] + str(NUMBERS[answer_tuple[0]])

    buttons[3].text = answer_coords
    buttons[3].button.configure(text=answer_coords)

def cell_click(i, j):
    global turn
    if cells[i][j].color == 'empty':
        if turn % 2 == 0:
            buttons[1].story.append(LETTERS[j] + str(i + 1))
            buttons[1].log.append('black: ' + LETTERS[j] + str(i + 1))
            buttons[1].button.configure(text='\n'.join(buttons[1].log))            
            cells[i][j].button.configure(text='black')            
            cells[i][j].button.configure(image=pics[give_pic(i, j, 1)])        
            cells[i][j].color = 'black'
            turn += 1
        else:
            buttons[1].story.append(LETTERS[j] + str(i + 1))
            buttons[1].log.append('white: ' + LETTERS[j] + str(i + 1))
            buttons[1].button.configure(text='\n'.join(buttons[1].log))
            cells[i][j].button.configure(text='white')
            cells[i][j].button.configure(image=pics[give_pic(i, j, -1)])
            cells[i][j].color = 'white'
            turn += 1
        predictShow()

    elif cells[i][j].color == 'black':
        buttons[1].story.pop()        
        buttons[1].log.pop()
        buttons[1].button.configure(text='\n'.join(buttons[1].log))        
        cells[i][j].button.configure(text='empty')
        cells[i][j].button.configure(image=pics[give_pic(i, j)])
        cells[i][j].color = 'empty'
        turn -= 1

    elif cells[i][j].color == 'white':
        buttons[1].story.pop()
        buttons[1].log.pop()
        buttons[1].button.configure(text='\n'.join(buttons[1].log))        
        cells[i][j].button.configure(text='empty')
        cells[i][j].button.configure(image=pics[give_pic(i, j)]) 
        cells[i][j].color = 'empty'
        turn -= 1
        
# <<<<<<<<<<<<<<<<<<<<<<<<<


def refresh(start=False):
    global buttons
    global cells
    global turn
    turn = 0
    if not start:
        buttons[1].story = []
        buttons[1].log = []
    
    for i in range(15):
        btn = Button(text=LETTERS[i])
        btn.place(x=480 + 40 * i, y=95, width=20, height=20)
    for i in range(15):
        btn = Button(text=str(i + 1))
        btn.place(x=445, y=130 + 40 * i, width=20, height=20)     
    for i in range(TABLE_HEIGHT):
        for j in range(TABLE_WIDTH):
            pic_name = give_pic(i, j)

            cells[i][j] = Cell(i, j,
                               Button(root,
                                      command=lambda i=i, j=j: cell_click(i, j),
                                      image=pics[pic_name],
                                      text='empty'),
                               color='empty')
            cells[i][j].button.place(x=470 + (CELL_WIDTH) * j,
                                     y=120 + (CELL_HEIGHT) * i,
                                     width=CELL_WIDTH,
                                     height=CELL_HEIGHT)
    btnRefresh = Button(root,
                        command=lambda: refresh(), #<<<<<<<<<<<<<<<<<<<<<<<<< FIX
                        text='Restart')
    buttons[0] = btnRefresh
    buttons[0].place(x=320,
                     y=140,
                     width=120,
                     height=60)
    
    btnLast = Button(root,
                     text='Log')
    buttons[1] = BattleLog(btnLast, [])
    buttons[1].button.place(x=320,
                            y=220,
                            width=120,
                            height=300)
    
    btnGivePredict = Button(root, 
                            command=lambda: predictShow(),
                            text='Prediction')
    buttons[2] = btnGivePredict
    buttons[2].place(x=1100,
                     y=140,
                     width=120,
                     height=60)
  
    btnPrediction = Button(root,
                           text='Answer')
    buttons[3] = Texter(btnPrediction, '')
    buttons[3].button.place(x=1100,
                     y=220,
                     width=120,
                     height=60)    

In [ ]:
root = Tk()
root.title('Renju')


pictureEmpty = ImageTk.PhotoImage(file=pictureEmptyDir)
pictureEmptyUp = ImageTk.PhotoImage(file=pictureEmptyUpDir)
pictureEmptyDown = ImageTk.PhotoImage(file=pictureEmptyDownDir)
pictureEmptyRight = ImageTk.PhotoImage(file=pictureEmptyRightDir)
pictureEmptyLeft = ImageTk.PhotoImage(file=pictureEmptyLeftDir)
pictureEmptySpecial = ImageTk.PhotoImage(file=pictureEmptySpecialDir)
pictureEmptyNW = ImageTk.PhotoImage(file=pictureEmptyNWDir)
pictureEmptyNE = ImageTk.PhotoImage(file=pictureEmptyNEDir)
pictureEmptySW = ImageTk.PhotoImage(file=pictureEmptySWDir)
pictureEmptySE = ImageTk.PhotoImage(file=pictureEmptySEDir)

pictureBlack = ImageTk.PhotoImage(file=pictureBlackDir)
pictureBlackUp = ImageTk.PhotoImage(file=pictureBlackUpDir)
pictureBlackDown = ImageTk.PhotoImage(file=pictureBlackDownDir)
pictureBlackRight = ImageTk.PhotoImage(file=pictureBlackRightDir)
pictureBlackLeft = ImageTk.PhotoImage(file=pictureBlackLeftDir)
pictureBlackNW = ImageTk.PhotoImage(file=pictureBlackNWDir)
pictureBlackNE = ImageTk.PhotoImage(file=pictureBlackNEDir)
pictureBlackSW = ImageTk.PhotoImage(file=pictureBlackSWDir)
pictureBlackSE = ImageTk.PhotoImage(file=pictureBlackSEDir)

pictureWhite = ImageTk.PhotoImage(file=pictureWhiteDir)
pictureWhiteUp = ImageTk.PhotoImage(file=pictureWhiteUpDir)
pictureWhiteDown = ImageTk.PhotoImage(file=pictureWhiteDownDir)
pictureWhiteRight = ImageTk.PhotoImage(file=pictureWhiteRightDir)
pictureWhiteLeft = ImageTk.PhotoImage(file=pictureWhiteLeftDir)
pictureWhiteNW = ImageTk.PhotoImage(file=pictureWhiteNWDir)
pictureWhiteNE = ImageTk.PhotoImage(file=pictureWhiteNEDir)
pictureWhiteSW = ImageTk.PhotoImage(file=pictureWhiteSWDir)
pictureWhiteSE = ImageTk.PhotoImage(file=pictureWhiteSEDir)

pics = {
    'emptyM': pictureEmpty,
    'emptyN': pictureEmptyUp,
    'emptyS': pictureEmptyDown,
    'emptyE': pictureEmptyRight,
    'emptyW': pictureEmptyLeft,
    'emptyNE': pictureEmptyNE,
    'emptySE': pictureEmptySE,
    'emptySW': pictureEmptySW,
    'emptyNW': pictureEmptyNW,
    'emptySpec': pictureEmptySpecial,
    
    'blackM': pictureBlack,
    'blackN': pictureBlackUp,
    'blackS': pictureBlackDown,
    'blackE': pictureBlackRight,
    'blackW': pictureBlackLeft,
    'blackNE': pictureBlackNE,
    'blackSE': pictureBlackSE,
    'blackSW': pictureBlackSW,
    'blackNW': pictureBlackNW,
    'blackSpec': pictureBlack,
    
    'whiteM': pictureWhite,
    'whiteN': pictureWhiteUp,
    'whiteS': pictureWhiteDown,
    'whiteE': pictureWhiteRight,
    'whiteW': pictureWhiteLeft,
    'whiteNE': pictureWhiteNE,
    'whiteSE': pictureWhiteSE,
    'whiteSW': pictureWhiteSW,
    'whiteNW': pictureWhiteNW,
    'whiteSpec': pictureWhite
       }

can = Canvas(root, width=root.winfo_screenwidth(), height=root.winfo_screenheight())


refresh(start=True)
can.pack()
root.mainloop()